# OpenStreetMap RAG pipeline

> [OpenStreetMap](https://www.openstreetmap.org/) is a map of the world, created by people like you and free to use under an open license.

OpenStreetMap has already provided [Overpass Api](https://wiki.openstreetmap.org/wiki/Overpass_API) queries

notebook 会展示：基础 OSM 查询 → LLM summarization → Agent + tools


## Setup Environment
- Install `haystack-ai`, `osm-integration-haystack`
- Setup `OPENAI_API_KEY`



In [ ]:
!pip install -q haystack-ai osm-integration-haystack

## Add Your API Keys

In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" in os.environ:
    del os.environ["OPENAI_API_KEY"]

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")


Enter OpenAI API key:··········


## Part1: Setup Environment

这个部分是使用LLM agent处理OpenStreetMap数据的前置步骤。`OSMFetcher`来自'osm_integration_haystack'负责通过封装的`Overpass API` queries...

### 1.1 From Name [String] to Coordination [Tuple]

这个部分示范了使用[Nominatim](https://nominatim.org/)来decode为具体坐标。这个子章节主要说明了数据输入的一种alternative。

In [ ]:
!pip install -q geopy

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="haystack-osm-cookbook-demo")

# Geo-decoding a name string into geocode
location_name = "saints peter and paul's catholic church"
location = geolocator.geocode(location_name)

print(f"Query: {location_name}")
print(f"Latitude:  {location.latitude}")
print(f"Longitude: {location.longitude}")
print(f"Display name: {location.address}")


Query: saints peter and paul's catholic church
Latitude:  51.8989077
Longitude: -8.4743188
Display name: Saints Peter and Paul's Catholic Church, Carey's Lane, The Marsh, Centre B ED, Cork, County Cork, Munster, T12 FH27, Éire / Ireland


实际的Geocoding/decoding往往还需要考虑歧义和命中率等问题，预处理并非是本教程的重点。在一般的map-based软件实践中，出于对精度的考虑，后端接受的往往是一组实际的(latitude, longitude) tuple。在此我们就直接使用Saints Peter and Paul's Catholic Church的坐标，并尝试寻找200米内最近的咖啡店。

In [ ]:
from osm_integration_haystack import OSMFetcher

CENTER = (51.8989077, -8.4743188)  # (lat, lon)
RADIUS_M = 1000

In [ ]:
osm_fetcher = OSMFetcher(
        preset_center=CENTER,  # Cork, Ireland
        preset_radius_m=RADIUS_M,  # 200m radius
        target_osm_types=["node"],  # Only search nodes
        target_osm_tags=["amenity"],  # Search amenity types
        maximum_query_mb=2,  # Limit query size
        overpass_timeout=20
    )

In [ ]:
result = osm_fetcher.run()     # Haystack component 标准接口
documents = result["documents"]

Current Query:

        [out:json][timeout:20][maxsize:2000000];
        (
            node[amenity](around:1000,51.8989077,-8.4743188);
        );
        out geom;
        
Status: 200
Response: {
  "version": 0.6,
  "generator": "Overpass API 0.7.62.8 e802775f",
  "osm3s": {
    "timestamp_osm_base": "2025-11-15T15:10:27Z",
    "copyright": "The data included in this document is from www.ope...
[OSM_Doc_Converter] Reading Raw OSM GeoJson...
[OSM_Doc_Converter] Loaded 955 entries.
[OSM_Doc_Converter] Batch-processing data cleaning.


In [ ]:
from pprint import pprint

first_doc = documents[0]
print("📄 type:", type(first_doc))

print("\n--- content ---")
print(first_doc.content)

print("\n--- meta keys ---")
print(list(first_doc.meta.keys()))

print("\n--- full meta ---")
pprint(first_doc.meta)


📄 type: <class 'haystack.dataclasses.document.Document'>

--- content ---
Restaurant: Koto, Carey's Lane, 6-7, T12 FH27. Tags: opening_hours=Mo-Su 12:00-22:00

--- meta keys ---
['source', 'osm_id', 'osm_type', 'lat', 'lon', 'name', 'category', 'tags', 'tags_norm', 'address', 'distance_m']

--- full meta ---
{'address': {'housenumber': '6-7',
             'postcode': 'T12 FH27',
             'street': "Carey's Lane"},
 'category': 'restaurant',
 'distance_m': 27.86087599824802,
 'lat': 51.8990101,
 'lon': -8.4739482,
 'name': 'Koto',
 'osm_id': 5203928867,
 'osm_type': 'node',
 'source': 'openstreetmap',
 'tags': {'amenity': 'restaurant',
          'contact:facebook': 'https://www.facebook.com/KotoCork/',
          'contact:instagram': 'https://www.instagram.com/kotocork',
          'cuisine': 'asian',
          'email': 'info@koto.ie',
          'opening_hours': 'Mo-Su 12:00-22:00',
          'phone': '+353-21-4274172',
          'smoking': 'no',
          'website': 'https://koto.ie/

In [ ]:
def preview_documents(docs, limit=5):
    print(f"Previewing first {min(len(docs), limit)} documents:\n")

    for i, doc in enumerate(docs[:limit], start=1):
        name = doc.meta.get("name", "Unknown")
        category = doc.meta.get("category", "Unknown")
        distance = doc.meta.get("distance_m", 0.0)
        lat = doc.meta.get("lat")
        lon = doc.meta.get("lon")

        print(f"{i}. {name}")
        print(f"   Type: {category}")
        print(f"   Distance: {distance:.1f} m")
        print(f"   Location: ({lat}, {lon})")
        print(f"   Content: {doc.content[:120]}{'...' if len(doc.content) > 120 else ''}")
        print()

preview_documents(documents, limit=5)


Previewing first 5 documents:

1. Koto
   Type: restaurant
   Distance: 27.9 m
   Location: (51.8990101, -8.4739482)
   Content: Restaurant: Koto, Carey's Lane, 6-7, T12 FH27. Tags: opening_hours=Mo-Su 12:00-22:00

2. Dukes
   Type: cafe
   Distance: 28.7 m
   Location: (51.8991234, -8.474089)
   Content: Cafe: Dukes, Carey's Lane, 4, Cork.

3. Soba Asian Street Food
   Type: fast_food
   Distance: 30.1 m
   Location: (51.8989516, -8.4738856)
   Content: Fast_food: Soba Asian Street Food.

4. OffBeat Donuts
   Type: fast_food
   Distance: 35.1 m
   Location: (51.8990968, -8.4739097)
   Content: Fast_food: OffBeat Donuts, French Church Street, 17, Cork.

5. Burritos and Blues
   Type: fast_food
   Distance: 43.6 m
   Location: (51.899271, -8.4745565)
   Content: Fast_food: Burritos and Blues, Paul Street, 9, Cork. Tags: opening_hours=Mo-We 12:00-20:00; Th-Sa 12:00-21:00; Su 13:00-...



## Part2: Pipeline to look for the nearest coffee shop

In [ ]:
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage
from haystack.utils import Secret

In [ ]:
prompt_template = [
    ChatMessage.from_system(
        "You are a geographic information assistant. "
        "Based on the provided OpenStreetMap data, help the user find nearby places that match the user's query."
    ),
    ChatMessage.from_user(
        """
        User location: {{ user_location }}
        Search radius: {{ radius }}m
        User query: {{ query }}

        Available location data:
        {% for document in documents %}
        - {{ document.content }}
          Location: ({{ document.meta.lat }}, {{ document.meta.lon }})
          Distance: {{ document.meta.distance_m }}m
          Type: {{ document.meta.category }}
        {% endfor %}

        Please:
        1. Find all locations that are relevant to the user's query
        2. Sort them by distance
        3. Recommend the nearest 3 locations
        4. Provide a short description for each

        Please respond in English.
        """
    ),
]

prompt_builder = ChatPromptBuilder(
    template=prompt_template,
    required_variables=["user_location", "radius", "query", "documents"], # optional, depends on what your pipeline requires
)


In [ ]:
llm = OpenAIChatGenerator(
    api_key=Secret.from_env_var("OPENAI_API_KEY"),
    model="gpt-4o-mini",
)

In [ ]:
coffee_pipeline = Pipeline()
coffee_pipeline.add_component("osm_fetcher", osm_fetcher)
coffee_pipeline.add_component("prompt_builder", prompt_builder)
coffee_pipeline.add_component("llm", llm)

# documents to prompt_builder
coffee_pipeline.connect("osm_fetcher.documents", "prompt_builder.documents")
# ChatPromptBuilder output toward prompt(List[ChatMessage]) as llm.messages
coffee_pipeline.connect("prompt_builder.prompt", "llm.messages")


🚅 Components
  - osm_fetcher: OSMFetcher
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
🛤️ Connections
  - osm_fetcher.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (list[ChatMessage])

In [ ]:
search_query = "coffee shop"

In [ ]:
user_location = "Cork, Ireland"
radius = 1000

result = coffee_pipeline.run(
    {
        "osm_fetcher": {},
        "prompt_builder": {
            "user_location": user_location,
            "radius": radius,
            "query": search_query,
        },
    }
)

reply = result["llm"]["replies"][0]
print("Role:", reply.role)
print("\nAssistant reply:\n")
print(reply.text)


Current Query:

        [out:json][timeout:20][maxsize:2000000];
        (
            node[amenity](around:1000,51.8989077,-8.4743188);
        );
        out geom;
        
Status: 200
Response: {
  "version": 0.6,
  "generator": "Overpass API 0.7.62.8 e802775f",
  "osm3s": {
    "timestamp_osm_base": "2025-11-15T15:11:30Z",
    "copyright": "The data included in this document is from www.ope...
[OSM_Doc_Converter] Reading Raw OSM GeoJson...
[OSM_Doc_Converter] Loaded 955 entries.
[OSM_Doc_Converter] Batch-processing data cleaning.
Role: ChatRole.ASSISTANT

Assistant reply:

Based on your query for coffee shops in Cork within a 1000m radius from your location, here are the nearest three options:

1. **Dukes, Carey's Lane, 4, Cork**  
   - **Distance:** 28.70m  
   - **Description:** A cozy café located on Carey's Lane, perfect for grabbing a quick coffee or enjoying a light snack in a relaxed atmosphere.

2. **Plus & Minus, Cork**  
   - **Distance:** 45.59m  
   - **Description:** T

## Part 3 : Planning an afternoon itinerary with an Agent and OSM tools



In [177]:
from osm_integration_haystack import OSMFetcher

ITINERARY_CENTER = (51.898403, -8.473978)  # (lat, lon)
ITINERARY_RADIUS_M = 1000                  # 1000m 半径

itinerary_fetcher = OSMFetcher(
    preset_center=ITINERARY_CENTER,
    preset_radius_m=ITINERARY_RADIUS_M,
    target_osm_types=["node"],
    target_osm_tags=[
        "amenity",   # cafe / bar / pub / restaurant / place_of_worship
        "tourism",   # 景点、博物馆
        "leisure",   # 公园等
    ],
    maximum_query_mb=4,
    overpass_timeout=30,
)

itinerary_fetcher


Inputs:
  - center: Optional[Tuple[float, float]]
  - radius_m: Optional[int]
Outputs:
  - documents: List[Document]